In [ ]:
!pip install pandas
!pip install nltk
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:00


In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk



nltk.download('punkt')
nltk.download('stopwords')


data_pekerjaan = pd.read_csv('set_kerja.csv')


ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):

    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    stemmed_tokens = [ps.stem(token) for token in filtered_tokens]
    return " ".join(stemmed_tokens)

def rekomendasi_pekerjaan(jawaban_pengguna, data_pekerjaan):
    rekomendasi = []

    jawaban_disabilitas = jawaban_pengguna[0].lower()
    jawaban_keahlian = jawaban_pengguna[2].lower()
    jawaban_lokasi = jawaban_pengguna[1].lower()

    for index, pekerjaan in data_pekerjaan.iterrows():
        eligibility = pekerjaan['Eligibility'].lower()
        job_requirement = preprocess_text(pekerjaan['Job Requirement'].lower())
        job_description = preprocess_text(pekerjaan['Job Description'].lower())
        lokasi = pekerjaan['Lokasi'].lower()

        disabilitas_similarity = fuzz.token_set_ratio(jawaban_disabilitas, eligibility)
        keahlian_similarity = max(fuzz.token_set_ratio(jawaban_keahlian, job_requirement),
                                  fuzz.token_set_ratio(jawaban_keahlian, job_description))
        lokasi_similarity = fuzz.token_set_ratio(jawaban_lokasi, lokasi)

        if disabilitas_similarity > 50 and keahlian_similarity > 50 and lokasi_similarity > 50:
            rekomendasi.append(pekerjaan)

    return rekomendasi


disabilitas_tertentu = ['Cacat Kaki', 'Cacat Tangan', 'Tuna Wicara', 'Tuna Rungu', 'Tuna Netra']


lokasi_terurut = sorted(data_pekerjaan['Lokasi'].unique())


keahlian_terurut = sorted(data_pekerjaan['Keahlian'].unique())


dropdown_disabilitas = widgets.Dropdown(
    options=['Pilih Disabilitas...'] + disabilitas_tertentu,
    description='Apa jenis disabilitas yang Anda miliki?',
    value='Pilih Disabilitas...'
)

dropdown_lokasi = widgets.Dropdown(
    options=['Pilih Lokasi...'] + lokasi_terurut,
    description='Dimana Anda berlokasi saat ini?',
    value='Pilih Lokasi...'
)

dropdown_keahlian = widgets.Dropdown(
    options=['Pilih Keahlian...'] + keahlian_terurut,
    description='Anda memiliki keahlian apa?',
    value='Pilih Keahlian...'
)

display(dropdown_disabilitas, dropdown_lokasi, dropdown_keahlian)


def dapatkan_rekomendasi(b, data):
    jawaban_pengguna = [dropdown_disabilitas.value, dropdown_lokasi.value, dropdown_keahlian.value]
    if 'Pilih' in jawaban_pengguna:
        print("Silakan pilih semua opsi terlebih dahulu.")
        return

    hasil_rekomendasi = rekomendasi_pekerjaan(jawaban_pengguna, data)
    if hasil_rekomendasi:
        print("\nRekomendasi Pekerjaan untuk Anda:")
        for pekerjaan in hasil_rekomendasi:
            print(pekerjaan[['Nama Pekerjaan', 'Job Description', 'Eligibility', 'Job Requirement', 'Gaji', 'Lokasi', 'Nama Perusahaan', 'Keahlian']])
    else:
        print("Maaf, tidak ada pekerjaan yang cocok dengan kriteria Anda.")

tombol = widgets.Button(description="Dapatkan Rekomendasi")
tombol.on_click(lambda b: dapatkan_rekomendasi(b, data_pekerjaan))
display(tombol)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Dropdown(description='Apa jenis disabilitas yang Anda miliki?', options=('Pilih Disabilitas...', 'Cacat Kaki',…

Dropdown(description='Dimana Anda berlokasi saat ini?', options=('Pilih Lokasi...', 'Badung, Bali', 'Batanghar…

Dropdown(description='Anda memiliki keahlian apa?', options=('Pilih Keahlian...', 'Administrasi ', 'Berkomunik…

Button(description='Dapatkan Rekomendasi', style=ButtonStyle())


Rekomendasi Pekerjaan untuk Anda:
Nama Pekerjaan                                               Seniman
Job Description    Seorang seniman pembuat vas tanah liat biasany...
Eligibility                       Cacat Kaki/Tuna Rungu/Tuna Wicara 
Job Requirement    Memiliki keterampilan dalam penggunaan alat da...
Gaji                                                    4 - 5 jt/bln
Lokasi                                                 Gianyar, Bali
Nama Perusahaan                                       Pengrajin Mode
Keahlian                                              Keahlian Seni 
Name: 8, dtype: object
Nama Pekerjaan                                               Seniman
Job Description    Anda akan menggunakan teknik tradisional atau ...
Eligibility                       Cacat Kaki/Tuna Rungu/Tuna Wicara 
Job Requirement    Kreativitas yang tinggi dan kemampuan untuk me...
Gaji                                                    4 - 5 jt/bln
Lokasi                                       